## In this experiment, we will train and test 3 models each with CNN and LSTM.

## In these models we will be training on one dataset and testing on another dataset

#### Importing the necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import tensorflow as tf
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tensorflow import keras
from sklearn.metrics import classification_report,confusion_matrix
from keras.models import Sequential
from keras import layers
import string

[nltk_data] Downloading package stopwords to /home/kvp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


#### Reading the dataset

In [2]:
import os,glob
folder_path ='MT'
i=0
MT_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        MT_sents.extend(f.read().split("\n"))
MT_sents=list(set(MT_sents))

folder_path ='OC'
i=0
OC_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        OC_sents.extend(f.read().split("\n"))
OC_sents=list(set(OC_sents))

folder_path ='PE'
i=0
PE_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        PE_sents.extend(f.read().split("\n"))
PE_sents=list(set(PE_sents))
        
folder_path ='VG'
i=0
VG_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        VG_sents.extend(f.read().split("\n"))
VG_sents=list(set(VG_sents))
        
folder_path ='WD'
i=0
WD_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        WD_sents.extend(f.read().split("\n"))
WD_sents=list(set(WD_sents))

folder_path ='WTP'
i=0
WTP_sents=[]
for filename in glob.glob(os.path.join(folder_path, '*.train')):
    with open(filename, 'r',encoding='utf8') as f:
        WTP_sents.extend(f.read().split("\n"))
WTP_sents=list(set(WTP_sents))

In [4]:
print(len(MT_sents))
print(len(OC_sents))
print(len(PE_sents))
print(len(VG_sents))
print(len(WD_sents))
print(len(WTP_sents))

450
8529
7052
2577
3873
8411


#### Data preprocessing and cleaning

In [5]:
MT_s=[]
MT_l=[]
for i in MT_sents:
    if len(i)>1:
        MT_s.append(i.split("\t")[0])
        MT_l.append(int(i.split("\t")[1]))

        
OC_s=[]
OC_l=[]
for i in OC_sents:
    if len(i)>1:
        OC_s.append(i.split("\t")[0])
        OC_l.append(int(i.split("\t")[1]))

        
PE_s=[]
PE_l=[]
for i in PE_sents:
    if len(i)>1:
        PE_s.append(i.split("\t")[0])
        PE_l.append(int(i.split("\t")[1]))
        

VG_s=[]
VG_l=[]
for i in VG_sents:
    if len(i)>1:
        VG_s.append(i.split("\t")[0])
        VG_l.append(int(i.split("\t")[1]))
        
        
WD_s=[]
WD_l=[]
for i in WD_sents:
    if len(i)>1:
        WD_s.append(i.split("\t")[0])
        WD_l.append(int(i.split("\t")[1]))
        
        
WTP_s=[]
WTP_l=[]
for i in WTP_sents:
    if len(i)>1:
        WTP_s.append(i.split("\t")[0])
        WTP_l.append(int(i.split("\t")[1]))

In [7]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text


#### Converting the datastet into a dataframe

In [8]:
df_MT=pd.DataFrame({'sentence':MT_s,'label':MT_l})
df_OC=pd.DataFrame({'sentence':OC_s,'label':OC_l})
df_PE=pd.DataFrame({'sentence':PE_s,'label':PE_l})
df_VG=pd.DataFrame({'sentence':VG_s,'label':VG_l})
df_WD=pd.DataFrame({'sentence':WD_s,'label':WD_l})
df_WTP=pd.DataFrame({'sentence':WTP_s,'label':WTP_l})

In [9]:
df_MT['sentence'] = df_MT['sentence'].map(lambda x: clean_text(x))
df_OC['sentence'] = df_OC['sentence'].map(lambda x: clean_text(x))
df_PE['sentence'] = df_PE['sentence'].map(lambda x: clean_text(x))
df_VG['sentence'] = df_VG['sentence'].map(lambda x: clean_text(x))
df_WD['sentence'] = df_WD['sentence'].map(lambda x: clean_text(x))
df_WTP['sentence'] = df_WTP['sentence'].map(lambda x: clean_text(x))

#### Showing the dataset

In [11]:
df_MT.head()

,sentence,label
0,year hous shortag berlin use tempelhof feld of...,0
1,fine leav dog wast behind admittedli seem rath...,0
2,would except watch quiz show tend watch big br...,0
3,long unclear work fund rather spent therapi on...,0
4,therefor offer accord audienc rate would danger,0


In [12]:
df_OC.head()

,sentence,label
0,hope everyon wonder holiday season,0
1,quick warm paint today,0
2,hug tight,0
3,glad xxxx,0
4,pleas disturb still four five kid knock ring d...,0


In [13]:
df_PE.head()

,sentence,label
0,languag extinct discuss view,0
1,also advertis product take place mass media po...,0
2,believ nowaday peopl alway confront problem wh...,0
3,artifici fertil catalyz capac inch farmland or...,0
4,time children share everyth school project eve...,0


In [14]:
df_VG.head()

,sentence,label
0,one use year though,0
1,would unfortun men spend great deal money stat...,1
2,lawyer martin defend case mafia therefor clean...,1
3,ref 2003 ukhl,0
4,god exist,0


In [15]:
df_WD.head()

,sentence,label
0,readi,0
1,start high qualiti fair educ focus techniqu wo...,0
2,depend person,0
3,doubt fund full time academ advisor,0
4,guess respect realli matter,0


In [18]:
df_WTP.head()

,sentence,label
0,sinc chang convent discuss seem even precipit,0
1,origin cerebr materi,0
2,militari suicid pictur alreadi two black white...,1
3,think much pov issu rais recent overblown subt...,1
4,clearli specif forbid drive traffic social net...,0


In [19]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

### Cross Domain Experiments Using CNN model

#### For each dataset, we will be training 20 models and in each model we will discard certain negative samples inorder to tackle the data imbalance and have no loss of information

#### Training on MT dataset and testing on OC dataset

In [20]:
f1_MTOC=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_MT[df_MT["label"]==1]
    df_negative=df_MT[df_MT["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))  
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences_train=df["sentence"].values
    y_train=df["label"].values
    sentences_test=df_OC.sentence.values
    y_test=df_OC.label.values
    
    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences_train)

    X_train=tokenizer.texts_to_sequences(sentences_train)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=30

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)

    def create_embedding_matrix(filepath,word_index,embedding_dim):
        vocab_size=len(word_index)+1
        embedding_matrix=np.zeros((vocab_size,embedding_dim))
        with open(filepath,encoding="utf8") as f:
            for line in f:
                word,*vector=line.split()
                if word in word_index:
                    idx=word_index[word]
                    embedding_matrix[idx]=np.array(vector,dtype=np.float32)[:embedding_dim]
        return embedding_matrix

    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    from keras.models import Sequential
    from keras import layers
    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=15,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    y_pred_bool = np.argmax(y_pred, axis=1)
    print(classification_report(y_test, y_pred.round()))
    report=classification_report(y_test, y_pred.round(),output_dict=True)
    f1_MTOC.append(report["macro avg"]["f1-score"])

Model 1
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/15
224/224 [==============================] - 1s 2ms/step - loss: 0.6957 - accuracy: 0.4643
Epoch 2/15
224/224 [==============================] - 0s 446us/step - loss: 0.6493 - accuracy: 0.8750
Epoch 3/15
224/224 [==============================] - 0s 427us/step - loss: 0.5677 - accuracy: 0.9777
Epoch 4/15
224/224 [==============================] - 0s 520us/step - loss: 0.3937 - accuracy: 0.9955
Epoch 5/15
224/224 [==============================] - 0s 475us/step - loss: 0.1650 - accuracy: 1.0000
Epoch 6/15
224/224 [==============================] - 0s 665us/step - loss: 0.0453 - accuracy: 1.0000
Epoch 7/15
224/224 [==============================] - 0s 481us/step - loss: 0.0163 - accuracy: 1.0000
Epoch 8/15
224/224 [==============================] - 0s 420us/step - loss: 0.0087 - accuracy: 1.0000
Epoch 9/15
224/224 [==============================] - 0s 454us/step - loss: 0.0056 

Epoch 1/15
224/224 [==============================] - 0s 1ms/step - loss: 0.6944 - accuracy: 0.4911
Epoch 2/15
224/224 [==============================] - 0s 425us/step - loss: 0.6229 - accuracy: 0.9643
Epoch 3/15
224/224 [==============================] - 0s 412us/step - loss: 0.5081 - accuracy: 0.9866
Epoch 4/15
224/224 [==============================] - 0s 449us/step - loss: 0.2915 - accuracy: 0.9955
Epoch 5/15
224/224 [==============================] - 0s 445us/step - loss: 0.0899 - accuracy: 1.0000
Epoch 6/15
224/224 [==============================] - 0s 409us/step - loss: 0.0273 - accuracy: 1.0000
Epoch 7/15
224/224 [==============================] - 0s 439us/step - loss: 0.0185 - accuracy: 0.9955
Epoch 8/15
224/224 [==============================] - 0s 439us/step - loss: 0.0106 - accuracy: 1.0000
Epoch 9/15
224/224 [==============================] - 0s 603us/step - loss: 0.0055 - accuracy: 1.0000
Epoch 10/15
224/224 [==============================] - 0s 595us/step - loss: 0.0038 

224/224 [==============================] - 0s 330us/step - loss: 0.0055 - accuracy: 1.0000
Epoch 10/15
224/224 [==============================] - 0s 326us/step - loss: 0.0034 - accuracy: 1.0000
Epoch 11/15
224/224 [==============================] - 0s 332us/step - loss: 0.0022 - accuracy: 1.0000
Epoch 12/15
224/224 [==============================] - 0s 339us/step - loss: 0.0017 - accuracy: 1.0000
Epoch 13/15
224/224 [==============================] - 0s 329us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 14/15
224/224 [==============================] - 0s 333us/step - loss: 0.0010 - accuracy: 1.0000
Epoch 15/15
8528/8528 [==============================] - 0s 52us/step
              precision    recall  f1-score   support

           0       0.91      0.41      0.56      7826
           1       0.07      0.52      0.13       702

    accuracy                           0.42      8528
   macro avg       0.49      0.47      0.35      8528
weighted avg       0.84      0.42      0.53      852

In [23]:
print(f"The Macro F1 score of the model is {sum(f1_MTOC)/len(f1_MTOC)}")

The Macro F1 score of the model is 0.4302899355099217


#### Training on the PE dataset and testing on the VG dataset

In [24]:
f1_PEVG=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_PE[df_PE["label"]==1]
    df_negative=df_PE[df_PE["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences_train=df["sentence"].values
    y_train=df["label"].values
    sentences_test=df_VG.sentence.values
    y_test=df_VG.label.values
    
    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences_train)

    X_train=tokenizer.texts_to_sequences(sentences_train)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=30

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)

    def create_embedding_matrix(filepath,word_index,embedding_dim):
        vocab_size=len(word_index)+1
        embedding_matrix=np.zeros((vocab_size,embedding_dim))
        with open(filepath,encoding="utf8") as f:
            for line in f:
                word,*vector=line.split()
                if word in word_index:
                    idx=word_index[word]
                    embedding_matrix[idx]=np.array(vector,dtype=np.float32)[:embedding_dim]
        return embedding_matrix

    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    from keras.models import Sequential
    from keras import layers
    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=15,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    y_pred_bool = np.argmax(y_pred, axis=1)
    print(classification_report(y_test, y_pred.round()))
    report=classification_report(y_test, y_pred.round(),output_dict=True)
    f1_PEVG.append(report["macro avg"]["f1-score"])

Model 1
Epoch 1/15
4186/4186 [==============================] - 4s 926us/step - loss: 0.6244 - accuracy: 0.6374
Epoch 2/15
4186/4186 [==============================] - 4s 863us/step - loss: 0.4440 - accuracy: 0.7979
Epoch 3/15
4186/4186 [==============================] - 4s 879us/step - loss: 0.2436 - accuracy: 0.9092
Epoch 4/15
4186/4186 [==============================] - 4s 877us/step - loss: 0.0993 - accuracy: 0.9677
Epoch 5/15
4186/4186 [==============================] - 4s 857us/step - loss: 0.0400 - accuracy: 0.9883
Epoch 6/15
4186/4186 [==============================] - 4s 855us/step - loss: 0.0226 - accuracy: 0.9959
Epoch 7/15
4186/4186 [==============================] - 4s 884us/step - loss: 0.0124 - accuracy: 0.9979
Epoch 8/15
4186/4186 [==============================] - 4s 848us/step - loss: 0.0099 - accuracy: 0.9990
Epoch 9/15
4186/4186 [==============================] - 4s 855us/step - loss: 0.0082 - accuracy: 0.9988
Epoch 10/15
4186/4186 [==============================] -

4186/4186 [==============================] - 4s 856us/step - loss: 0.0120 - accuracy: 0.9986
Epoch 8/15
4186/4186 [==============================] - 4s 849us/step - loss: 0.0127 - accuracy: 0.9986
Epoch 9/15
4186/4186 [==============================] - 4s 839us/step - loss: 0.0091 - accuracy: 0.9986
Epoch 10/15
4186/4186 [==============================] - 4s 845us/step - loss: 0.0086 - accuracy: 0.9986
Epoch 11/15
4186/4186 [==============================] - 4s 847us/step - loss: 0.0070 - accuracy: 0.9990
Epoch 12/15
4186/4186 [==============================] - 4s 848us/step - loss: 0.0072 - accuracy: 0.9983
Epoch 13/15
4186/4186 [==============================] - 4s 854us/step - loss: 0.0093 - accuracy: 0.9981
Epoch 14/15
4186/4186 [==============================] - 4s 838us/step - loss: 0.0081 - accuracy: 0.9976
Epoch 15/15
2576/2576 [==============================] - 1s 291us/step
              precision    recall  f1-score   support

           0       0.80      0.80      0.80     

4186/4186 [==============================] - 4s 944us/step - loss: 0.0128 - accuracy: 0.9981
Epoch 14/15
4186/4186 [==============================] - 4s 935us/step - loss: 0.0105 - accuracy: 0.9981
Epoch 15/15
2576/2576 [==============================] - 1s 463us/step
              precision    recall  f1-score   support

           0       0.80      0.80      0.80      2029
           1       0.25      0.24      0.25       547

    accuracy                           0.68      2576
   macro avg       0.52      0.52      0.52      2576
weighted avg       0.68      0.68      0.68      2576

Model 18
Epoch 1/15
4186/4186 [==============================] - 6s 1ms/step - loss: 0.6174 - accuracy: 0.6601
Epoch 2/15
4186/4186 [==============================] - 4s 1ms/step - loss: 0.4212 - accuracy: 0.8134
Epoch 3/15
4186/4186 [==============================] - 4s 1ms/step - loss: 0.2210 - accuracy: 0.9169
Epoch 4/15
4186/4186 [==============================] - 4s 989us/step - loss: 0.0870 - ac

In [25]:
print(f"The Macro F1 score of the model is {sum(f1_PEVG)/len(f1_PEVG)}")

The Macro F1 score of the model is 0.5301513729209858


#### Training on the WD dataset and testing on the WTP dataset

In [26]:
f1_WDWTP=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_WD[df_WD["label"]==1]
    df_negative=df_WD[df_WD["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    sentences_train=df["sentence"].values
    y_train=df["label"].values
    sentences_test=df_WTP.sentence.values
    y_test=df_WTP.label.values
    
    tokenizer=Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(sentences_train)

    X_train=tokenizer.texts_to_sequences(sentences_train)
    X_test=tokenizer.texts_to_sequences(sentences_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=30

    X_train=pad_sequences(X_train,padding='post',maxlen=maxlen)
    X_test=pad_sequences(X_test,padding='post',maxlen=maxlen)

    def create_embedding_matrix(filepath,word_index,embedding_dim):
        vocab_size=len(word_index)+1
        embedding_matrix=np.zeros((vocab_size,embedding_dim))
        with open(filepath,encoding="utf8") as f:
            for line in f:
                word,*vector=line.split()
                if word in word_index:
                    idx=word_index[word]
                    embedding_matrix[idx]=np.array(vector,dtype=np.float32)[:embedding_dim]
        return embedding_matrix

    embedding_dim=50
    embedding_matrix=create_embedding_matrix('glove.6B/glove.6B.50d.txt',tokenizer.word_index,embedding_dim)


    from keras.models import Sequential
    from keras import layers
    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    
    history = model.fit(X_train, y_train,
                        epochs=15,
                        batch_size=10)
    y_pred = model.predict(X_test, batch_size=64, verbose=1)
    y_pred_bool = np.argmax(y_pred, axis=1)
    print(classification_report(y_test, y_pred.round()))
    report=classification_report(y_test, y_pred.round(),output_dict=True)
    f1_WDWTP.append(report["macro avg"]["f1-score"])

Model 1
Epoch 1/15
422/422 [==============================] - 1s 2ms/step - loss: 0.6850 - accuracy: 0.5427
Epoch 2/15
422/422 [==============================] - 1s 1ms/step - loss: 0.5902 - accuracy: 0.8057
Epoch 3/15
422/422 [==============================] - 0s 781us/step - loss: 0.3970 - accuracy: 0.8768
Epoch 4/15
422/422 [==============================] - 0s 662us/step - loss: 0.1757 - accuracy: 0.9739
Epoch 5/15
422/422 [==============================] - 0s 617us/step - loss: 0.0603 - accuracy: 0.9905
Epoch 6/15
422/422 [==============================] - 0s 796us/step - loss: 0.0316 - accuracy: 0.9905
Epoch 7/15
422/422 [==============================] - 0s 779us/step - loss: 0.0153 - accuracy: 0.9953
Epoch 8/15
422/422 [==============================] - 0s 646us/step - loss: 0.0093 - accuracy: 1.0000
Epoch 9/15
422/422 [==============================] - 0s 491us/step - loss: 0.0060 - accuracy: 1.0000
Epoch 10/15
422/422 [==============================] - 0s 599us/step - loss: 0

422/422 [==============================] - 0s 483us/step - loss: 0.0046 - accuracy: 1.0000
Epoch 10/15
422/422 [==============================] - 0s 485us/step - loss: 0.0030 - accuracy: 1.0000
Epoch 11/15
422/422 [==============================] - 0s 484us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 12/15
422/422 [==============================] - 0s 485us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 13/15
422/422 [==============================] - 0s 483us/step - loss: 0.0015 - accuracy: 1.0000
Epoch 14/15
422/422 [==============================] - 0s 486us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 15/15
8410/8410 [==============================] - 1s 166us/step
              precision    recall  f1-score   support

           0       0.87      0.80      0.83      7275
           1       0.15      0.22      0.18      1135

    accuracy                           0.72      8410
   macro avg       0.51      0.51      0.50      8410
weighted avg       0.77      0.72      0.74      84

Epoch 1/15
422/422 [==============================] - 1s 2ms/step - loss: 0.6910 - accuracy: 0.4882
Epoch 2/15
422/422 [==============================] - 0s 566us/step - loss: 0.6612 - accuracy: 0.5498
Epoch 3/15
422/422 [==============================] - 0s 578us/step - loss: 0.5749 - accuracy: 0.7583
Epoch 4/15
422/422 [==============================] - 0s 521us/step - loss: 0.3381 - accuracy: 0.9313
Epoch 5/15
422/422 [==============================] - 0s 539us/step - loss: 0.1369 - accuracy: 0.9692
Epoch 6/15
422/422 [==============================] - 0s 526us/step - loss: 0.0660 - accuracy: 0.9882
Epoch 7/15
422/422 [==============================] - 0s 562us/step - loss: 0.0437 - accuracy: 0.9905
Epoch 8/15
422/422 [==============================] - 0s 538us/step - loss: 0.0322 - accuracy: 0.9953
Epoch 9/15
422/422 [==============================] - 0s 738us/step - loss: 0.0191 - accuracy: 0.9976
Epoch 10/15
422/422 [==============================] - 0s 769us/step - loss: 0.0085 

In [27]:
print(f"The Macro F1 score of the model is {sum(f1_WDWTP)/len(f1_WDWTP)}")

The Macro F1 score of the model is 0.5017448759619938


## Cross Domain Experiments Using LSTM

#### Training on MT dataset and testing on OC Dataset

In [31]:
f1_LSTMMTOC=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_MT[df_MT["label"]==1]
    df_negative=df_MT[df_MT["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x_train=df.sentence.values
    y_train=df.label.values
    x_test=df_OC.sentence.values
    y_test=df_OC.label.values

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x_train)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_LSTMMTOC.append(result["macro avg"]["f1-score"])

Model 1
Model: "sequential_74"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_74 (Embedding)     (None, 20, 50)            49300     
_________________________________________________________________
lstm_27 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_28 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_13 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_145 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_146 (Dense)            (None, 1)                 9         
Total params: 72,037
Trainable params: 72,037
Non-trainable params: 0
_________________________________________

224/224 [==============================] - 0s 1ms/step - loss: 0.3941 - accuracy: 0.8705
Epoch 14/20
224/224 [==============================] - 0s 1ms/step - loss: 0.3376 - accuracy: 0.8884
Epoch 15/20
224/224 [==============================] - 0s 1ms/step - loss: 0.3256 - accuracy: 0.9107
Epoch 16/20
224/224 [==============================] - 0s 1ms/step - loss: 0.3190 - accuracy: 0.9018
Epoch 17/20
224/224 [==============================] - 0s 1ms/step - loss: 0.3455 - accuracy: 0.8795
Epoch 18/20
224/224 [==============================] - 0s 1ms/step - loss: 0.2929 - accuracy: 0.9196
Epoch 19/20
224/224 [==============================] - 0s 1ms/step - loss: 0.2879 - accuracy: 0.9107
Epoch 20/20
224/224 [==============================] - 0s 1ms/step - loss: 0.2718 - accuracy: 0.9196
              precision    recall  f1-score   support

           0       0.92      0.88      0.90      7826
           1       0.12      0.19      0.15       702

    accuracy                           0

Epoch 1/20
224/224 [==============================] - 2s 10ms/step - loss: 0.6949 - accuracy: 0.5045
Epoch 2/20
224/224 [==============================] - 0s 1ms/step - loss: 0.6919 - accuracy: 0.5357
Epoch 3/20
224/224 [==============================] - 0s 2ms/step - loss: 0.6883 - accuracy: 0.5089
Epoch 4/20
224/224 [==============================] - 0s 1ms/step - loss: 0.6961 - accuracy: 0.4732
Epoch 5/20
224/224 [==============================] - 0s 2ms/step - loss: 0.6862 - accuracy: 0.5580
Epoch 6/20
224/224 [==============================] - 0s 1ms/step - loss: 0.5924 - accuracy: 0.7009
Epoch 7/20
224/224 [==============================] - 0s 2ms/step - loss: 0.6076 - accuracy: 0.6964
Epoch 8/20
224/224 [==============================] - 0s 1ms/step - loss: 0.5633 - accuracy: 0.7321
Epoch 9/20
224/224 [==============================] - 0s 1ms/step - loss: 0.4879 - accuracy: 0.7812
Epoch 10/20
224/224 [==============================] - 0s 1ms/step - loss: 0.4457 - accuracy: 0.808

Model: "sequential_86"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_86 (Embedding)     (None, 20, 50)            49000     
_________________________________________________________________
lstm_51 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_52 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_25 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_169 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_170 (Dense)            (None, 1)                 9         
Total params: 71,737
Trainable params: 71,737
Non-trainable params: 0
_________________________________________________

224/224 [==============================] - 0s 1ms/step - loss: 0.3968 - accuracy: 0.8527
Epoch 14/20
224/224 [==============================] - 0s 1ms/step - loss: 0.3793 - accuracy: 0.8527
Epoch 15/20
224/224 [==============================] - 0s 1ms/step - loss: 0.3448 - accuracy: 0.8616
Epoch 16/20
224/224 [==============================] - 0s 1ms/step - loss: 0.3293 - accuracy: 0.8839
Epoch 17/20
224/224 [==============================] - 0s 1ms/step - loss: 0.3212 - accuracy: 0.8973
Epoch 18/20
224/224 [==============================] - 0s 1ms/step - loss: 0.2983 - accuracy: 0.9107
Epoch 19/20
224/224 [==============================] - 0s 1ms/step - loss: 0.2980 - accuracy: 0.9152
Epoch 20/20
224/224 [==============================] - 0s 1ms/step - loss: 0.2961 - accuracy: 0.9107
              precision    recall  f1-score   support

           0       0.92      0.88      0.90      7826
           1       0.11      0.17      0.13       702

    accuracy                           0

Epoch 1/20
224/224 [==============================] - 3s 12ms/step - loss: 0.6943 - accuracy: 0.5357
Epoch 2/20
224/224 [==============================] - 0s 1ms/step - loss: 0.6936 - accuracy: 0.5134
Epoch 3/20
224/224 [==============================] - 0s 1ms/step - loss: 0.6924 - accuracy: 0.4866
Epoch 4/20
224/224 [==============================] - 0s 2ms/step - loss: 0.6900 - accuracy: 0.5446
Epoch 5/20
224/224 [==============================] - 0s 1ms/step - loss: 0.6900 - accuracy: 0.5402
Epoch 6/20
224/224 [==============================] - 0s 1ms/step - loss: 0.6807 - accuracy: 0.5402
Epoch 7/20
224/224 [==============================] - 0s 1ms/step - loss: 0.5884 - accuracy: 0.7500
Epoch 8/20
224/224 [==============================] - 0s 1ms/step - loss: 0.5753 - accuracy: 0.7500
Epoch 9/20
224/224 [==============================] - 0s 1ms/step - loss: 0.5193 - accuracy: 0.7634
Epoch 10/20
224/224 [==============================] - 0s 1ms/step - loss: 0.4711 - accuracy: 0.825

In [32]:
print(f"The Macro F1 score of the model is {sum(f1_LSTMMTOC)/len(f1_LSTMMTOC)}")

The Macro F1 score of the model is 0.5186095697709512


#### Training on PE dataset and testing on VG dataset

In [33]:
f1_LSTMPEVG=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_PE[df_PE["label"]==1]
    df_negative=df_PE[df_PE["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x_train=df.sentence.values
    y_train=df.label.values
    x_test=df_VG.sentence.values
    y_test=df_VG.label.values

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x_train)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_LSTMPEVG.append(result["macro avg"]["f1-score"])

Model 1
Model: "sequential_94"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_94 (Embedding)     (None, 20, 50)            183250    
_________________________________________________________________
lstm_67 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_68 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_33 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_185 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_186 (Dense)            (None, 1)                 9         
Total params: 205,987
Trainable params: 205,987
Non-trainable params: 0
_______________________________________

Epoch 1/20
4186/4186 [==============================] - 8s 2ms/step - loss: 0.6748 - accuracy: 0.5650
Epoch 2/20
4186/4186 [==============================] - 6s 1ms/step - loss: 0.6201 - accuracy: 0.6641
Epoch 3/20
4186/4186 [==============================] - 6s 2ms/step - loss: 0.6051 - accuracy: 0.6792
Epoch 4/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.6006 - accuracy: 0.6808
Epoch 5/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.6005 - accuracy: 0.6806
Epoch 6/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.5995 - accuracy: 0.6868
Epoch 7/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.5941 - accuracy: 0.6856
Epoch 8/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.5877 - accuracy: 0.6871
Epoch 9/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.5883 - accuracy: 0.6880
Epoch 10/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.577

Model: "sequential_101"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_101 (Embedding)    (None, 20, 50)            179900    
_________________________________________________________________
lstm_81 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_82 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_40 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_199 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_200 (Dense)            (None, 1)                 9         
Total params: 202,637
Trainable params: 202,637
Non-trainable params: 0
______________________________________________

Epoch 1/20
4186/4186 [==============================] - 10s 2ms/step - loss: 0.6665 - accuracy: 0.5800
Epoch 2/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.6163 - accuracy: 0.6706
Epoch 3/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.6084 - accuracy: 0.6818
Epoch 4/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.6059 - accuracy: 0.6861
Epoch 5/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.6012 - accuracy: 0.6887
Epoch 6/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.6004 - accuracy: 0.6854
Epoch 7/20
4186/4186 [==============================] - 8s 2ms/step - loss: 0.5984 - accuracy: 0.6863
Epoch 8/20
4186/4186 [==============================] - 8s 2ms/step - loss: 0.5905 - accuracy: 0.6892
Epoch 9/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.5880 - accuracy: 0.6918
Epoch 10/20
4186/4186 [==============================] - 6s 1ms/step - loss: 0.58

Model: "sequential_108"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_108 (Embedding)    (None, 20, 50)            184000    
_________________________________________________________________
lstm_95 (LSTM)               (None, 20, 50)            20200     
_________________________________________________________________
lstm_96 (LSTM)               (None, 10)                2440      
_________________________________________________________________
dropout_47 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_213 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_214 (Dense)            (None, 1)                 9         
Total params: 206,737
Trainable params: 206,737
Non-trainable params: 0
______________________________________________

Epoch 1/20
4186/4186 [==============================] - 10s 2ms/step - loss: 0.6752 - accuracy: 0.5674
Epoch 2/20
4186/4186 [==============================] - 6s 1ms/step - loss: 0.6196 - accuracy: 0.6718
Epoch 3/20
4186/4186 [==============================] - 6s 1ms/step - loss: 0.6051 - accuracy: 0.6818
Epoch 4/20
4186/4186 [==============================] - 6s 1ms/step - loss: 0.6074 - accuracy: 0.6887
Epoch 5/20
4186/4186 [==============================] - 6s 1ms/step - loss: 0.6044 - accuracy: 0.6854
Epoch 6/20
4186/4186 [==============================] - 6s 1ms/step - loss: 0.5980 - accuracy: 0.6847
Epoch 7/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.5937 - accuracy: 0.6949
Epoch 8/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.5925 - accuracy: 0.6904
Epoch 9/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.5877 - accuracy: 0.6928
Epoch 10/20
4186/4186 [==============================] - 7s 2ms/step - loss: 0.58

In [34]:
print(f"The Macro F1 score of the model is {sum(f1_LSTMPEVG)/len(f1_LSTMPEVG)}")

The Macro F1 score of the model is 0.5472902758288595


#### Training on WD dataset and testing on WTP dataset

In [35]:
f1_LSTMWDWTP=[]
for i in range(0,20):
    print(f"Model {i+1}")
    df_positive=df_WD[df_WD["label"]==1]
    df_negative=df_WD[df_WD["label"]==0]
    df_negative=df_negative.sample(n=len(df_positive))
    frames=[df_positive,df_negative]
    df=pd.concat(frames)
    x_train=df.sentence.values
    y_train=df.label.values
    x_test=df_WTP.sentence.values
    y_test=df_WTP.label.values

    tokenizer = Tokenizer(num_words=100)
    tokenizer.fit_on_texts(x_train)
    xtrain= tokenizer.texts_to_sequences(x_train)
    xtest= tokenizer.texts_to_sequences(x_test)

    vocab_size=len(tokenizer.word_index)+1

    maxlen=20
    xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
    xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

    embedding_dim=50
    model=Sequential()
    model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
    model.add(layers.LSTM(units=50,return_sequences=True))
    model.add(layers.LSTM(units=10))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
    model.summary()
    model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=True)
    ypred=model.predict(xtest)
    ypred[ypred>0.5]=1 
    ypred[ypred<=0.5]=0 
    print(classification_report(y_test, ypred))
    result=classification_report(y_test, ypred,output_dict=True)
    f1_LSTMWDWTP.append(result["macro avg"]["f1-score"])

Model 1
Model: "sequential_114"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_114 (Embedding)    (None, 20, 50)            63900     
_________________________________________________________________
lstm_107 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_108 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_53 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_225 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_226 (Dense)            (None, 1)                 9         
Total params: 86,637
Trainable params: 86,637
Non-trainable params: 0
________________________________________

422/422 [==============================] - 1s 2ms/step - loss: 0.4068 - accuracy: 0.8720
Epoch 14/20
422/422 [==============================] - 1s 2ms/step - loss: 0.3793 - accuracy: 0.8886
Epoch 15/20
422/422 [==============================] - 1s 2ms/step - loss: 0.3692 - accuracy: 0.8863
Epoch 16/20
422/422 [==============================] - 1s 2ms/step - loss: 0.3612 - accuracy: 0.8886
Epoch 17/20
422/422 [==============================] - 1s 2ms/step - loss: 0.3628 - accuracy: 0.8910
Epoch 18/20
422/422 [==============================] - 1s 2ms/step - loss: 0.3502 - accuracy: 0.8886
Epoch 19/20
422/422 [==============================] - 1s 1ms/step - loss: 0.3743 - accuracy: 0.8720
Epoch 20/20
422/422 [==============================] - 1s 2ms/step - loss: 0.3873 - accuracy: 0.8720
              precision    recall  f1-score   support

           0       0.87      0.84      0.85      7275
           1       0.16      0.19      0.17      1135

    accuracy                           0

Epoch 1/20
422/422 [==============================] - 5s 12ms/step - loss: 0.6933 - accuracy: 0.5095
Epoch 2/20
422/422 [==============================] - 1s 2ms/step - loss: 0.6878 - accuracy: 0.5498
Epoch 3/20
422/422 [==============================] - 1s 2ms/step - loss: 0.6082 - accuracy: 0.7156
Epoch 4/20
422/422 [==============================] - 1s 2ms/step - loss: 0.5465 - accuracy: 0.7725
Epoch 5/20
422/422 [==============================] - 1s 2ms/step - loss: 0.4902 - accuracy: 0.8009
Epoch 6/20
422/422 [==============================] - 1s 2ms/step - loss: 0.4791 - accuracy: 0.8270
Epoch 7/20
422/422 [==============================] - 1s 2ms/step - loss: 0.4605 - accuracy: 0.8389
Epoch 8/20
422/422 [==============================] - 1s 2ms/step - loss: 0.4694 - accuracy: 0.8365
Epoch 9/20
422/422 [==============================] - 1s 2ms/step - loss: 0.4292 - accuracy: 0.8365
Epoch 10/20
422/422 [==============================] - 1s 2ms/step - loss: 0.4230 - accuracy: 0.843

Model: "sequential_126"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_126 (Embedding)    (None, 20, 50)            65750     
_________________________________________________________________
lstm_131 (LSTM)              (None, 20, 50)            20200     
_________________________________________________________________
lstm_132 (LSTM)              (None, 10)                2440      
_________________________________________________________________
dropout_65 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_249 (Dense)            (None, 8)                 88        
_________________________________________________________________
dense_250 (Dense)            (None, 1)                 9         
Total params: 88,487
Trainable params: 88,487
Non-trainable params: 0
________________________________________________

422/422 [==============================] - 1s 2ms/step - loss: 0.4044 - accuracy: 0.8626
Epoch 14/20
422/422 [==============================] - 1s 2ms/step - loss: 0.3782 - accuracy: 0.8720
Epoch 15/20
422/422 [==============================] - 1s 2ms/step - loss: 0.3725 - accuracy: 0.8697
Epoch 16/20
422/422 [==============================] - 1s 2ms/step - loss: 0.3768 - accuracy: 0.8815
Epoch 17/20
422/422 [==============================] - 1s 2ms/step - loss: 0.3567 - accuracy: 0.8768
Epoch 18/20
422/422 [==============================] - 1s 2ms/step - loss: 0.3672 - accuracy: 0.8886
Epoch 19/20
422/422 [==============================] - 1s 2ms/step - loss: 0.3597 - accuracy: 0.8886
Epoch 20/20
422/422 [==============================] - 1s 2ms/step - loss: 0.3475 - accuracy: 0.8934
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      7275
           1       0.16      0.16      0.16      1135

    accuracy                           0

Epoch 1/20
422/422 [==============================] - 6s 14ms/step - loss: 0.6973 - accuracy: 0.4479
Epoch 2/20
422/422 [==============================] - 1s 2ms/step - loss: 0.6929 - accuracy: 0.5071
Epoch 3/20
422/422 [==============================] - 1s 2ms/step - loss: 0.6934 - accuracy: 0.5166
Epoch 4/20
422/422 [==============================] - 1s 2ms/step - loss: 0.6506 - accuracy: 0.6351
Epoch 5/20
422/422 [==============================] - 1s 2ms/step - loss: 0.5868 - accuracy: 0.7180
Epoch 6/20
422/422 [==============================] - 1s 2ms/step - loss: 0.5138 - accuracy: 0.7820
Epoch 7/20
422/422 [==============================] - 1s 2ms/step - loss: 0.5060 - accuracy: 0.7773
Epoch 8/20
422/422 [==============================] - 1s 2ms/step - loss: 0.4758 - accuracy: 0.8033
Epoch 9/20
422/422 [==============================] - 1s 2ms/step - loss: 0.4785 - accuracy: 0.8365
Epoch 10/20
422/422 [==============================] - 1s 2ms/step - loss: 0.4402 - accuracy: 0.838

In [36]:
print(f"The Macro F1 score of the model is {sum(f1_LSTMWDWTP)/len(f1_LSTMWDWTP)}")

The Macro F1 score of the model is 0.5092333757325936
